In [1]:
# header files
# please note that in this notebook we are performing survival analysis using 'sksurv' package on the D1 cohort
# D1 cohort - Cervix cancer, pretreatment scans treated with chemotherapy after surgery
%matplotlib inline
import glob
import csv
import numpy as np
import pandas as pd
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.svm import HingeLossSurvivalSVM
from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]
print("Header files loaded!")

Header files loaded!


In [4]:
# training: 95 TCGA ovarian cancer cases treated with chemotherapy 
# this block consists of four variables: train_features, train_y, train_event, train_survival_time loaded from data.csv
# train_features: 34 features used, combination of collagen and til
# train_y: each value in the array is (event, survival_time) where event is 'True' (if death or reccurence occured) or 'False' (no event occured) and survival time is the time from disease being diagnosed to event
# train_event: event is 'True' (if death or reccurence occured) or 'False' (no event occured)
# train_survival_time: survival time is the time from disease being diagnosed to event
train_features = []
train_y = []
train_event = []
train_survival_time = []

flag = -1
with open("../../../data/data.csv", newline='', encoding = "ISO-8859-1") as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if flag == -1:
            flag = 1
            print(row)
        else:
            array = row
            if array[1] == "Train":
                f = []
                for index in range(6, 40):
                    f.append(float(array[index]))
                train_features.append(f)
                
                event = False
                if array[41] == "TRUE" or array[41] == "True" or array[41] == "true":
                    event = True
                train_y.append([event, float(array[40])])
                train_event.append(event)
                train_survival_time.append(float(array[40]))
print(len(train_features))
print(len(train_y))
print(len(train_event))
print(len(train_survival_time))

['Patient ID', 'Data split', 'Organ', 'Site', 'Treatment', 'Vital status', 'Feature 1(TIL)', 'Feature 2(TIL)', 'Feature 3(TIL)', 'Feature 4(TIL)', 'Feature 5(TIL)', 'Feature 6(TIL)', 'Feature 7(TIL)', 'Feature 8(Collagen)', 'Feature 9(Collagen)', 'Feature 10(Collagen)', 'Feature 11(Collagen)', 'Feature 12(Collagen)', 'Feature 13(Collagen)', 'Feature 14(Collagen)', 'Feature 15(Collagen)', 'Feature 16(Collagen)', 'Feature 17(Collagen)', 'Feature 18(Collagen)', 'Feature 19(Collagen)', 'Feature 20(Collagen)', 'Feature 21(Collagen)', 'Feature 22(Collagen)', 'Feature 23(Collagen)', 'Feature 24(Collagen)', 'Feature 25(Collagen)', 'Feature 26(Collagen)', 'Feature 27(Collagen)', 'Feature 28(Collagen)', 'Feature 29(Collagen)', 'Feature 30(Collagen)', 'Feature 31(Collagen)', 'Feature 32(Collagen)', 'Feature 33(Collagen)', 'Feature 34(Collagen)', 'OS', 'OS_event', 'PFS', 'PFS_event', 'Age', 'Stage', 'Tumor grade', 'Risk score', 'Binary risk score']
95
95
95
95


In [5]:
# this block basically converts the four variables created in previous block to numpy arrays 
# which will be used for training the model
train_features = np.array(train_features)
train_y = np.array(train_y)
train_event = np.array(train_event)
train_survival_time = np.array(train_survival_time)

In [6]:
# validation: 134 TCGA cervix cancer cases treated with chemotherapy 
# this block consists of four variables: train_features, train_y, train_event, train_survival_time loaded from data.csv
# train_features: 34 features used, combination of collagen and til
# train_y: each value in the array is (event, survival_time) where event is 'True' (if death or reccurence occured) or 'False' (no event occured) and survival time is the time from disease being diagnosed to event
# train_event: event is 'True' (if death or reccurence occured) or 'False' (no event occured)
# train_survival_time: survival time is the time from disease being diagnosed to event
test_features = []
test_y = []
test_event = []
test_survival_time = []
test_clinical_var_age = []
test_clinical_var_stage = []

flag = -1
with open("../../../data/data.csv", newline='', encoding = "ISO-8859-1") as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if flag == -1:
            flag = 1
            print(row)
        else:
            array = row
            if array[1] == "Validation" and array[2] == "Cervix" and array[4] == "Chemotherapy":
                f = []
                for index in range(6, 40):
                    f.append(float(array[index]))
                test_features.append(f)
                
                event = False
                if array[41] == "TRUE" or array[41] == "True" or array[41] == "true":
                    event = True
                test_y.append([event, float(array[40])])
                test_event.append(event)
                test_survival_time.append(float(array[40]))
                test_clinical_var_age.append(float(array[44]))
                test_clinical_var_stage.append(float(array[45]))
print(len(test_features))
print(len(test_y))
print(len(test_event))
print(len(test_survival_time))
print(len(test_clinical_var_age))
print(len(test_clinical_var_stage))

['Patient ID', 'Data split', 'Organ', 'Site', 'Treatment', 'Vital status', 'Feature 1(TIL)', 'Feature 2(TIL)', 'Feature 3(TIL)', 'Feature 4(TIL)', 'Feature 5(TIL)', 'Feature 6(TIL)', 'Feature 7(TIL)', 'Feature 8(Collagen)', 'Feature 9(Collagen)', 'Feature 10(Collagen)', 'Feature 11(Collagen)', 'Feature 12(Collagen)', 'Feature 13(Collagen)', 'Feature 14(Collagen)', 'Feature 15(Collagen)', 'Feature 16(Collagen)', 'Feature 17(Collagen)', 'Feature 18(Collagen)', 'Feature 19(Collagen)', 'Feature 20(Collagen)', 'Feature 21(Collagen)', 'Feature 22(Collagen)', 'Feature 23(Collagen)', 'Feature 24(Collagen)', 'Feature 25(Collagen)', 'Feature 26(Collagen)', 'Feature 27(Collagen)', 'Feature 28(Collagen)', 'Feature 29(Collagen)', 'Feature 30(Collagen)', 'Feature 31(Collagen)', 'Feature 32(Collagen)', 'Feature 33(Collagen)', 'Feature 34(Collagen)', 'OS', 'OS_event', 'PFS', 'PFS_event', 'Age', 'Stage', 'Tumor grade', 'Risk score', 'Binary risk score']
134
134
134
134
134
134


In [7]:
count = 0
for index in range(0, len(test_clinical_var_stage)):
    if test_clinical_var_stage[index] == 1.0:
        count += 1
print(count)

72


In [8]:
count = 0
for index in range(0, len(test_event)):
    if test_event[index] == False:
        count += 1
print(count)

106


In [9]:
print(np.mean(test_clinical_var_age))

48.38805970149254


In [10]:
# running survival model using the train and validation dataset defined above
# this block has four major variables: train_group, train_risk_scores, group, test_risk_scores
# train_group: binary risk score 1 or 0 for train dataset. 1: high risk group and 0: low risk group
# test_group: binary risk score 1 or 0 for test dataset. 1: high risk group and 0: low risk group
# train_risk_scores: risk scores for train dataset
# test_risk_scores: risk scores for test dataset
group = []
train_group = []
features_train = train_features
features_test = test_features
y_train = train_y
event_train, survival_time_train = train_event, train_survival_time
dt = dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
y_train = np.array([tuple(row) for row in y_train], dtype=dt)
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
features_train_df = pd.DataFrame(features_train)
features_test_df = pd.DataFrame(features_test)

estimator = CoxnetSurvivalAnalysis(l1_ratio=0.9, alpha_min_ratio=0.1)
#estimator = CoxPHSurvivalAnalysis()
estimator.fit(features_train_df, y_train)

score, _, _, _, _ = concordance_index_censored(test_event, test_survival_time, estimator.predict(features_test_df))
print("Test: " + str(score))
score, _, _, _, _ = concordance_index_censored(train_event, train_survival_time, estimator.predict(features_train_df))
print("Train: " + str(score))

# get risk scores
train_risk_scores = estimator.predict(features_train_df)
test_risk_scores = estimator.predict(features_test_df)

median = np.mean(train_risk_scores)
count_low = 0
count_high = 0
for index in range(0, len(train_risk_scores)):
    if train_risk_scores[index] > median:
        count_high += 1
        train_group.append(1)
    else:
        count_low += 1
        train_group.append(0)

count_low = 0
count_high = 0
for index in range(0, len(test_risk_scores)):
    if test_risk_scores[index] > median:
        count_high += 1
        group.append(1)
    else:
        count_low += 1
        group.append(0)

Test: 0.6957295373665481
Train: 0.7264375637972099


# RandomSurvivalForest
group = []
train_group = []
features_train = train_features
features_test = test_features
y_train = train_y
event_train, survival_time_train = train_event, train_survival_time
dt = dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
y_train = np.array([tuple(row) for row in y_train], dtype=dt)
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
features_train_df = pd.DataFrame(features_train)
features_test_df = pd.DataFrame(features_test)

estimator = RandomSurvivalForest(n_estimators=10).fit(features_train_df, y_train)
score, _, _, _, _ = concordance_index_censored(test_event, test_survival_time, estimator.predict(features_test_df))
print("Test: " + str(score))
score, _, _, _, _ = concordance_index_censored(train_event, train_survival_time, estimator.predict(features_train_df))
print("Train: " + str(score))

train_risk_scores = estimator.predict(features_train_df)
test_risk_scores = estimator.predict(features_test_df)

median = np.mean(train_risk_scores)
count_low = 0
count_high = 0
for index in range(0, len(train_risk_scores)):
    if train_risk_scores[index] > median:
        count_high += 1
        train_group.append(1)
    else:
        count_low += 1
        train_group.append(0)

count_low = 0
count_high = 0
for index in range(0, len(test_risk_scores)):
    if test_risk_scores[index] > median:
        count_high += 1
        group.append(1)
    else:
        count_low += 1
        group.append(0)

# HingeLossSurvivalSVM
group = []
train_group = []
features_train = train_features
features_test = test_features
y_train = train_y
event_train, survival_time_train = train_event, train_survival_time
dt = dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
y_train = np.array([tuple(row) for row in y_train], dtype=dt)
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
features_train_df = pd.DataFrame(features_train)
features_test_df = pd.DataFrame(features_test)

estimator = HingeLossSurvivalSVM().fit(features_train_df, y_train)
score, _, _, _, _ = concordance_index_censored(test_event, test_survival_time, estimator.predict(features_test_df))
print("Test: " + str(score))
score, _, _, _, _ = concordance_index_censored(train_event, train_survival_time, estimator.predict(features_train_df))
print("Train: " + str(score))

train_risk_scores = estimator.predict(features_train_df)
test_risk_scores = estimator.predict(features_test_df)

median = np.mean(train_risk_scores)
count_low = 0
count_high = 0
for index in range(0, len(train_risk_scores)):
    if train_risk_scores[index] > median:
        count_high += 1
        train_group.append(1)
    else:
        count_low += 1
        train_group.append(0)

count_low = 0
count_high = 0
for index in range(0, len(test_risk_scores)):
    if test_risk_scores[index] > median:
        count_high += 1
        group.append(1)
    else:
        count_low += 1
        group.append(0)

In [11]:
# this block prints values for variables 'test_event', 'test_survival_time' and 'group' defined above
# these values are used in the 'univariate.m' script to find the corresponding HR, p-values and 95% CI
# univariate analysis - CollaTIL features
a = []
for index in range(0, len(test_event)):
    if test_event[index] == False:
        a.append(0)
    else:
        a.append(1)
print(*a, sep="; ")

print(*test_survival_time, sep="; ")

print(*group, sep="; ")

0; 0; 1; 1; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 1; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 1; 1; 0; 1; 1; 0; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0
1688.0; 534.0; 1011.0; 157.0; 2234.0; 491.0; 6375.0; 1714.0; 275.0; 2226.0; 1427.0; 555.0; 1169.0; 1099.0; 454.0; 1221.0; 254.0; 1800.0; 1791.0; 1243.0; 555.0; 132.0; 494.0; 1456.0; 477.0; 2669.0; 621.0; 4467.0; 1756.0; 788.0; 4738.0; 11.0; 595.0; 1551.0; 45.0; 252.0; 1031.0; 1617.0; 617.0; 542.0; 0.0; 542.0; 491.0; 0.0; 1122.0; 623.0; 5271.0; 27.0; 284.0; 253.0; 4483.0; 524.0; 523.0; 284.0; 2114.0; 579.0; 483.0; 803.0; 567.0; 619.0; 471.0; 343.0; 965.0; 370.0; 372.0; 504.0; 607.0; 2859.0; 828.0; 0.0; 21.0; 1946.0; 830.0; 410.0; 919.0; 1367.0; 0.0; 2094.0; 34.0; 14.0; 74.0; 0.0; 469.0; 492.0;

In [12]:
t = []
for index in range(0, len(test_clinical_var_stage)):
    if test_clinical_var_stage[index] > 2:
        t.append(1)
    else:
        t.append(0)
print(*t, sep="; ")

0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 1; 0; 0; 0; 1; 1; 0; 0; 1; 0; 1; 1; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 1; 1; 0; 1; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 1; 1; 0; 0; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 0; 0; 1; 1; 0; 0; 0; 0; 1; 1; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0


In [13]:
# this block prints the indexes of prognostic features found from the trained model
# we will see the model finding 14 features, 6 from TIL and 8 from Collagen being predictive of OS
count = 0
for index1 in range(0, len(estimator.coef_)):
    flag = -1
    for index2 in range(99, 100):
        if estimator.coef_[index1][index2] > 0 or estimator.coef_[index1][index2] < 0:
            flag = 1
            print(str(index1) + " " + str(estimator.coef_[index1][index2]))
            break
    if flag == 1:
        count += 1
print()
print("Prognostic features count = " + str(count))

0 0.22206462793633197
1 -0.08147010241846915
2 0.4858650335897516
4 -1.1089285491767151
5 0.8914089361068985
6 1.1052263119682961
7 -0.4766343602255981
9 -0.1369853017725905
12 -0.10887558173143377
17 -0.3948680778152415
20 -0.9348001116567328
23 0.9366083479474306
30 0.6385060393592817
33 -2.036242024932007

Prognostic features count = 14


In [14]:
# this block prints values for variables 'test_event', 'test_survival_time' and 'group' defined above
# these values are used in the 'univariate.m' script to find the corresponding HR, p-values and 95% CI
# univariate analysis - age
a = []
for index in range(0, len(test_event)):
    if test_event[index] == False:
        a.append(0)
    else:
        a.append(1)
print(*a, sep="; ")

print(*test_survival_time, sep="; ")

g = []
for index in range(0, len(test_clinical_var_age)):
    if test_clinical_var_age[index] > 60:
        g.append(1)
    else:
        g.append(0)
print(*g, sep="; ")

0; 0; 1; 1; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 1; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 1; 1; 0; 1; 1; 0; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0
1688.0; 534.0; 1011.0; 157.0; 2234.0; 491.0; 6375.0; 1714.0; 275.0; 2226.0; 1427.0; 555.0; 1169.0; 1099.0; 454.0; 1221.0; 254.0; 1800.0; 1791.0; 1243.0; 555.0; 132.0; 494.0; 1456.0; 477.0; 2669.0; 621.0; 4467.0; 1756.0; 788.0; 4738.0; 11.0; 595.0; 1551.0; 45.0; 252.0; 1031.0; 1617.0; 617.0; 542.0; 0.0; 542.0; 491.0; 0.0; 1122.0; 623.0; 5271.0; 27.0; 284.0; 253.0; 4483.0; 524.0; 523.0; 284.0; 2114.0; 579.0; 483.0; 803.0; 567.0; 619.0; 471.0; 343.0; 965.0; 370.0; 372.0; 504.0; 607.0; 2859.0; 828.0; 0.0; 21.0; 1946.0; 830.0; 410.0; 919.0; 1367.0; 0.0; 2094.0; 34.0; 14.0; 74.0; 0.0; 469.0; 492.0;

In [15]:
# this block prints values for variables 'test_event', 'test_survival_time' and 'group' defined above
# these values are used in the 'univariate.m' script to find the corresponding HR, p-values and 95% CI
# univariate analysis - stage
a = []
for index in range(0, len(test_event)):
    if test_event[index] == False:
        a.append(0)
    else:
        a.append(1)
print(*a, sep="; ")

print(*test_survival_time, sep="; ")

g = []
for index in range(0, len(test_clinical_var_stage)):
    if test_clinical_var_stage[index] == 1 or test_clinical_var_stage[index] == 2:
        g.append(0)
    else:
        g.append(1)
print(*g, sep="; ")

0; 0; 1; 1; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 1; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 1; 1; 0; 1; 1; 0; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0
1688.0; 534.0; 1011.0; 157.0; 2234.0; 491.0; 6375.0; 1714.0; 275.0; 2226.0; 1427.0; 555.0; 1169.0; 1099.0; 454.0; 1221.0; 254.0; 1800.0; 1791.0; 1243.0; 555.0; 132.0; 494.0; 1456.0; 477.0; 2669.0; 621.0; 4467.0; 1756.0; 788.0; 4738.0; 11.0; 595.0; 1551.0; 45.0; 252.0; 1031.0; 1617.0; 617.0; 542.0; 0.0; 542.0; 491.0; 0.0; 1122.0; 623.0; 5271.0; 27.0; 284.0; 253.0; 4483.0; 524.0; 523.0; 284.0; 2114.0; 579.0; 483.0; 803.0; 567.0; 619.0; 471.0; 343.0; 965.0; 370.0; 372.0; 504.0; 607.0; 2859.0; 828.0; 0.0; 21.0; 1946.0; 830.0; 410.0; 919.0; 1367.0; 0.0; 2094.0; 34.0; 14.0; 74.0; 0.0; 469.0; 492.0;

In [16]:
# this block prints values for variables 'test_event', 'test_survival_time' and 'group' defined above
# these values are used in the 'univariate.m' script to find the corresponding HR, p-values and 95% CI
# multivariable analysis - age, stage, collatil
a = []
for index in range(0, len(test_event)):
    if test_event[index] == False:
        a.append(0)
    else:
        a.append(1)
print(*a, sep="; ")

print(*test_survival_time, sep="; ")

g_1 = []
for index in range(0, len(test_clinical_var_age)):
    if test_clinical_var_age[index] > 60:
        g_1.append(1)
    else:
        g_1.append(0)
g_2 = []
for index in range(0, len(test_clinical_var_stage)):
    if test_clinical_var_stage[index] == 1 or test_clinical_var_stage[index] == 2:
        g_2.append(np.abs(test_clinical_var_stage[index]))
    else:
        g_2.append(np.abs(test_clinical_var_stage[index]))
g = []
for index in range(0, len(group)):
    g.append([g_1[index], g_2[index], group[index]])
print(*g, sep="; ")

0; 0; 1; 1; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 1; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 0; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 1; 1; 0; 1; 1; 0; 1; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 0; 0; 0; 0; 0; 1; 1; 0; 0; 1; 1; 0; 0; 0; 0; 0; 0; 1; 0
1688.0; 534.0; 1011.0; 157.0; 2234.0; 491.0; 6375.0; 1714.0; 275.0; 2226.0; 1427.0; 555.0; 1169.0; 1099.0; 454.0; 1221.0; 254.0; 1800.0; 1791.0; 1243.0; 555.0; 132.0; 494.0; 1456.0; 477.0; 2669.0; 621.0; 4467.0; 1756.0; 788.0; 4738.0; 11.0; 595.0; 1551.0; 45.0; 252.0; 1031.0; 1617.0; 617.0; 542.0; 0.0; 542.0; 491.0; 0.0; 1122.0; 623.0; 5271.0; 27.0; 284.0; 253.0; 4483.0; 524.0; 523.0; 284.0; 2114.0; 579.0; 483.0; 803.0; 567.0; 619.0; 471.0; 343.0; 965.0; 370.0; 372.0; 504.0; 607.0; 2859.0; 828.0; 0.0; 21.0; 1946.0; 830.0; 410.0; 919.0; 1367.0; 0.0; 2094.0; 34.0; 14.0; 74.0; 0.0; 469.0; 492.0;